# A Geospatial API example - Colorado Surface Water Conditions

Check out the map <a href="https://data.colorado.gov/Water/DWR-Current-Surface-Water-Conditions-Map-Statewide/j5pc-4t32" target="_blank"> Colorado DWR Current Surface Water Conditions map</a>. 

We can access the data used to create the map at the link above using `Python`.

* The data that you will use are located here: <a href="https://data.colorado.gov/resource/j5pc-4t32.json" target="_blank">View JSON format data used to create surface water map.</a>
* And you can learn more about the data here: <a href="https://data.colorado.gov/Water/DWR-Current-Surface-Water-Conditions/4yw9-a5y6" target="_blank">View CO Current water surface </a>.

In [1]:
import requests
import folium
import urllib
from pandas.io.json import json_normalize
import pandas as pd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point

## Build the data URL
This data source uses SODA (Socrata Open Data API).
We want data from active stations in Boulder County, which we can request
by adding a query string to the Current Surface Water dataset identifier

In [33]:
# Construct URL
water_url = (
    "https://data.colorado.gov/resource/4yw9-a5y6.json"
    "?station_status=Active" 
    "&county=BOULDER")
water_url

<i class="fa fa-lightbulb-o" aria-hidden="true"></i> **ATTENTION WINDOWS USERS:**
We have noticed a bug where on windows machines, sometimes the https URL doesn't work.

Instead try the same url as above but using `http` instead:
`"http://data.colorado.gov/resource/j5pc-4t32.json?` 

In [4]:
data = requests.get(water_url)

In [5]:
type(data.json())

list

In [36]:
data.json()[0]

{'div': '1',
 'wd': '5',
 'county': 'BOULDER',
 'station_name': 'PECK-PELLA AUGMENTATION RETURN',
 'data_source': 'Co. Division of Water Resources',
 'dwr_abbrev': 'PECRTNCO',
 'station_status': 'Active',
 'station_type': 'Diversion',
 'date_time': '2022-03-31T08:00:00.000',
 'variable': 'DISCHRG',
 'amount': '0.00',
 'units': 'CFS',
 'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/PECRTNCO'},
 'location': {'latitude': '40.160705', 'longitude': '-105.168471'},
 'utm_x': '485654.0',
 'utm_y': '4445607.0',
 'location_accuracy': 'GPS'}

Remember that the JSON structure supports hierarchical data and can be NESTED. If you look at the structure of the `.json` file below, you can see that the location object, is nested with three sub objects:

* latitude
* longitude
* needs_recoding

Since `data.json()` is a `list` you can print out just the first few items of the list to look at your data as a sanity check.

In [35]:
data.json()[0]['location']

{'latitude': '40.160705', 'longitude': '-105.168471'}

## Convert JSON to Pandas DataFrame

Use the `json_normalize()` function from the `Pandas` library to convert this `JSON` data into a `Pandas DataFrame.` 

This function helps organize and flatten data into a semi-structed table. To learn more, check out the <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html" target="_blank">documentation</a>!

In [39]:
result = pd.json_normalize(data.json())
type(result)

pandas.core.frame.DataFrame

In [10]:
result.head()

pandas.core.frame.DataFrame

## Data Cleaning for Visualization
Notice that your longitude and latitude values are stored as strings. Do you think can create a map if these values are stored as strings?

In [12]:
result['location.latitude'][0]

'40.160705'

You can convert the strings to type `float` as follows. 

In [13]:
result['location.latitude'] = result['location.latitude'].astype(float)

In [14]:
result['location.latitude'][0]

40.160705

In [15]:
result['location.longitude'] = result['location.longitude'].astype(float)

In [16]:
result['location.longitude'][0]

-105.168471

Now that you have numeric values for mapping, make sure that are are no missing values. 

In [17]:
result.shape

(82, 21)

In [18]:
result['location.longitude'].isna().any()

False

In [19]:
result['location.latitude'].isna().any()

False

There are no `nan` values in this data. However, if there were, you could remove rows where a column has a `nan` value in a specific column with the following: 
`result_nonan = result.dropna(subset=['location.longitude', 'location.latitude'])`

## Data Visualization 

Convert `Pandas DataFrame` to a `Geopandas DataFrame` for easy mapping with `folium`.

In [25]:
geometry = [Point(xy) for xy in zip(result['location.longitude'], result['location.latitude'])]
crs = 'epsg:4326'
gdf = GeoDataFrame(result, crs=crs, geometry=geometry)

In [26]:
gdf.head()

,div,wd,county,station_name,data_source,dwr_abbrev,station_status,station_type,date_time,variable,...,utm_x,utm_y,location_accuracy,http_linkage.url,location.latitude,location.longitude,stage,usgs_station_id,flag,geometry
0,1,5,BOULDER,PECK-PELLA AUGMENTATION RETURN,Co. Division of Water Resources,PECRTNCO,Active,Diversion,2022-03-31T08:00:00.000,DISCHRG,...,485654.0,4445607.0,GPS,https://dwr.state.co.us/Tools/Stations/PECRTNCO,40.160705,-105.168471,NaN,NaN,NaN,POINT (-105.16847 40.16071)
1,1,5,BOULDER,"HIGHLAND DITCH AT LYONS, CO",Co. Division of Water Resources,HIGHLDCO,Active,Diversion,2022-03-31T09:30:00.000,DISCHRG,...,478215.6,4451655.5,User supplied,https://dwr.state.co.us/Tools/Stations/HIGHLDCO,40.215043,-105.256017,0.59,NaN,NaN,POINT (-105.25602 40.21504)
2,1,4,BOULDER,BOULDER-LARIMER BYPASS NEAR BERTHOUD,Co. Division of Water Resources,BOUBYPCO,Active,Diversion,2022-03-31T09:15:00.000,DISCHRG,...,485050.3,4456488.5,User supplied,https://dwr.state.co.us/Tools/Stations/BOUBYPCO,40.258726,-105.175817,-0.14,NaN,NaN,POINT (-105.17582 40.25873)
3,1,6,BOULDER,"BOULDER CREEK AT NORTH 75TH ST. NEAR BOULDER, CO",U.S. Geological Survey,BOCNORCO,Active,Stream,2022-03-31T09:15:00.000,DISCHRG,...,484742.7,4433505.4,User supplied,https://dwr.state.co.us/Tools/Stations/BOCNORCO,40.051655,-105.178883,NaN,06730200,NaN,POINT (-105.17888 40.05165)
4,1,6,BOULDER,SOUTH BOULDER & BEAR CREEK DITCH,Co. Division of Water Resources,SBBDITCO,Active,Diversion,2022-03-31T09:00:00.000,DISCHRG,...,479380.0,4422642.0,Digitized,https://dwr.state.co.us/Tools/Stations/SBBDITCO,39.953668,-105.241413,NaN,NaN,NaN,POINT (-105.24141 39.95367)


Then, you can plot the data using the folium functions `GeoJson()` and `add_to()` to add the data from the `Geopandas DataFrame` to the map object. 

In [49]:
m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(m)

m

We can also: 
  * cluster the markers
  * add a popup to each marker

In [48]:
from folium.plugins import MarkerCluster

m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(m)
for i, point in gdf.iterrows():
    folium.Marker(
        location=(point['location.latitude'], point['location.longitude']), 
        popup='Name: {} Precip: {}'.format(
            point.station_name, point.amount)
    ).add_to(marker_cluster)

m

<div class="notice--info" markdown="1">

## Additional Resources

* <a href="https://blog.datafiniti.co/4-reasons-you-should-use-json-instead-of-csv-2cac362f1943" target="_blank">Why you should use JSON instead of csv.</a>


* <a href="https://www.w3schools.com/js/js_json_intro.asp" target="_blank">W3schools JSON intro </a>

</div>